In [1]:
# Old method
from pkg_resources import working_set
from src.license_scanner.parse_license import parse_license

unknown_license = "UNKNOWN"
all_packages_old = []


for key in working_set.normalized_to_canonical_keys:
    """
    The metadata of a package is stored in "metadata" or "PKG-INFO"
    """
    try:
        package_name = working_set.normalized_to_canonical_keys[key]
        generic_name = package_name.replace("-", "_").lower()
        package = working_set.by_key[package_name]
        metadata_lines = package.get_metadata("METADATA").split("\n")
    except:
        try:
            metadata_lines = package.get_metadata("PKG-INFO").split("\n")
        except:
            all_packages_old.append([package_name, unknown_license, generic_name])
            continue

    license_arg_raw = ""
    license_classifier_raw = ""
    for line in metadata_lines:
        line: str = line
        if line.startswith("License-Expression: "):
            license_arg_raw = line.replace("License-Expression: ", "")
        if line.startswith("License: "):
            license_arg_raw = line.replace("License: ", "")
        if line.startswith("Classifier: License ::"):
            license_classifier_raw = line.split(" :: ")[-1]
            if license_classifier_raw.lower() in ["osi approved"]:
                license_classifier_raw = ""

    general_license = "NOT FOUND"
    license_arg = parse_license(license_arg_raw)
    license_classifier = parse_license(license_classifier_raw)

    # You can get the license from license argument or classifier
    if license_arg is not "" and license_arg != unknown_license:
        general_license = license_arg
    elif license_classifier:
        general_license = license_classifier
    else:
        general_license = unknown_license

    all_packages_old.append([package_name, general_license, generic_name])

all_packages_old.sort()
print("Number of packages found (old method):", len(all_packages_old))
all_packages_old

__phello__
_pyrepl
asyncio
collections
concurrent
ctypes
curses
dbm
email
encodings
ensurepip
html
http
idlelib
importlib
json
logging
multiprocessing
pathlib
pydoc_data
re
sqlite3
sysconfig
test
tkinter
tomllib
turtledemo
unittest
urllib
venv
wsgiref
xml
xmlrpc
zipfile
zoneinfo
IPython
PIL
_argon2_cffi_bindings
_distutils_hack
_pytest
_yaml
adodbapi
annotated_types
anyio
argon2
arrow
asttokens
async_lru
athena
attr
attrs
authlib
autocommand
babel
beartype
black
blackd


<>:43: SyntaxWarning: "is not" with 'str' literal. Did you mean "!="?
<>:43: SyntaxWarning: "is not" with 'str' literal. Did you mean "!="?
C:\Users\tom.nijhof\AppData\Local\Temp\ipykernel_6464\4015494294.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import working_set


bleach
blib2to3
bs4
build
cachecontrol
cachetools
catalogue
certifi
cffi
charset_normalizer
cleo
click
cloudpickle
colorama
coloredlogs
comm
confluent_kafka
contourpy
coverage
crashtest
cryptography
cv2
cycler
cymem
dateutil
debugpy
defusedxml
distlib
dns
dulwich
email_validator
exceptiongroup
executing
fastai
fastcore
fastdownload
fastjsonschema
fastprogress
fasttransform
filelock
findpython
flatbuffers
fontTools
fqdn
fsspec
functorch
future
h11
httpcore
httpx
httpx_auth
humanfriendly
idna
importlib_metadata
iniconfig
installer
invoke
ipykernel
ipywidgets
isapi
isoduration
isort
jedi
jinja2
joblib
json5
jsonschema
jsonschema_specifications
jupyter_client
jupyter_console
jupyter_core
jupyter_events
jupyter_lsp
jupyter_server
jupyter_server_terminals
jupyterlab
jupyterlab_pygments
jupyterlab_server
jupyterlab_widgets
keyring
kiwisolver
libfuturize
libpasteurize
license_scanner
markdown_it
markupsafe
matplotlib
matplotlib_inline
mdurl
mistune
more_itertools
mpmath
mrcfile
msgpack
murmurh

C:\Users\tom.nijhof\AppData\Local\Temp\ipykernel_6464\4015494294.py:43: SyntaxWarning: "is not" with 'str' literal. Did you mean "!="?
  if license_arg is not "" and license_arg != unknown_license:


[['annotated-types', None, 'annotated_types'],
 ['anyio', 'MIT license', 'anyio'],
 ['argon2-cffi', 'MIT license', 'argon2_cffi'],
 ['argon2-cffi-bindings', 'MIT license', 'argon2_cffi_bindings'],
 ['arrow', None, 'arrow'],
 ['asttokens', 'Apache license 2.0', 'asttokens'],
 ['async-lru', 'MIT license', 'async_lru'],
 ['athena-clients', None, 'athena_clients'],
 ['attrs', 'MIT license', 'attrs'],
 ['authlib', 'BSD 3-clause license', 'authlib'],
 ['autocommand', 'GNU Lesser General Public License v3.0 only', 'autocommand'],
 ['babel', 'BSD 3-clause license', 'babel'],
 ['backports.tarfile', None, 'backports.tarfile'],
 ['beartype', 'MIT license', 'beartype'],
 ['beautifulsoup4', 'MIT license', 'beautifulsoup4'],
 ['black', 'MIT license', 'black'],
 ['bleach', 'Apache license', 'bleach'],
 ['build', None, 'build'],
 ['cachecontrol', 'Apache license 2.0', 'cachecontrol'],
 ['cachetools', 'MIT license', 'cachetools'],
 ['catalogue', 'MIT license', 'catalogue'],
 ['certifi', 'Mozilla public

In [2]:
from copy_pkg_resources_2 import get_all_package_names

all_packages = get_all_package_names()
print("Number of packages found (new method):", len(all_packages))

Number of packages found (new method): 237


In [3]:
from typing import List
import importlib_metadata as im


def get_license(name: str) -> List[str]:
    all_licenses = []
    try:
        metas = im.metadata(name)
    except im.PackageNotFoundError:
        return [unknown_license]
    for key, value in metas.items():
        if key == "License-Expression":
            license = value
            if license:
                all_licenses.append(license)
        if key == "License":
            license = value
            if license:
                all_licenses.append(license)
        if key == "Classifier":
            if not value.startswith("License ::"):
                continue
            license = value.split("::")[-1]
            if license.strip() == "OSI Approved":
                continue
            if license:
                all_licenses.append(license)

    all_licenses = list(set(all_licenses))
    return all_licenses

In [4]:
all_packages_new = []
for name in get_all_package_names():
    generic_name = name.replace("-", "_").lower()
    all_packages_new.append([name, get_license(name), generic_name])


all_packages_new.sort()
for i in all_packages_new:
    if len(i[1]) != 1:
        print(i)

['MarkupSafe', ['Copyright 2010 Pallets\n\nRedistribution and use in source and binary forms, with or without\nmodification, are permitted provided that the following conditions are\nmet:\n\n1.  Redistributions of source code must retain the above copyright\n    notice, this list of conditions and the following disclaimer.\n\n2.  Redistributions in binary form must reproduce the above copyright\n    notice, this list of conditions and the following disclaimer in the\n    documentation and/or other materials provided with the distribution.\n\n3.  Neither the name of the copyright holder nor the names of its\n    contributors may be used to endorse or promote products derived from\n    this software without specific prior written permission.\n\nTHIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS\n"AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT\nLIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A\nPARTICULAR PURPOSE ARE DISCLAIMED. IN 

In [5]:
all_licenses = set()
for i in all_packages_new:
    for j in i[1]:
        all_licenses.add(j)

count = 0
for l in all_licenses:
    if parse_license(l) is None :
        print(repr(l))
        count += 1
print(f"Total unknown licenses: {count}")

'Modified BSD License'
'Copyright (c) 2015 Project Jupyter Contributors\nAll rights reserved.\n\nRedistribution and use in source and binary forms, with or without\nmodification, are permitted provided that the following conditions are met:\n\n1. Redistributions of source code must retain the above copyright notice, this\n   list of conditions and the following disclaimer.\n\n2. Redistributions in binary form must reproduce the above copyright notice,\n   this list of conditions and the following disclaimer in the documentation\n   and/or other materials provided with the distribution.\n\n3. Neither the name of the copyright holder nor the names of its\n   contributors may be used to endorse or promote products derived from\n   this software without specific prior written permission.\n\nTHIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"\nAND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE\nIMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS 

In [6]:
import pandas as pd

all_packages = pd.DataFrame(
    columns=[
        "package_name_old",
        "package_name_new",
        "package_name_general",
        "license_new",
        "license_old",
    ]
)

all_packages_new_2 = all_packages_new.copy()

# All old packages
for i in all_packages_old:
    new_package = []
    # Match with new packages
    for j in all_packages_new_2:
        if j[2] == i[2]:
            new_package = j
            all_packages_new_2.remove(j)

    new_row = pd.Series(
        {
            "package_name_old": i[0].strip(),
            "package_name_new": new_package[0].strip() if new_package else "",
            "package_name_general": i[2].strip(),
            "license_new": new_package[1] if new_package[1] else [],
            "license_old": i[1].strip() if i[1] else "",
        }
    )
    all_packages = pd.concat([all_packages, new_row.to_frame().T])

# Add new packages NOT matches an old one
for i in all_packages_new_2:

    new_row = pd.Series(
        {
            "package_name_new": i[0].strip(),
            "package_name_general": i[2].strip(),
            "license_new": i[1].strip(),
        }
    )
    all_packages = pd.concat([all_packages, new_row.to_frame().T])

all_packages = all_packages.fillna("")
all_packages.head()

AttributeError: 'list' object has no attribute 'strip'

In [ ]:
all_packages.to_csv("test_1.csv", index=False)

In [ ]:
print(
    f"Number of mismatching linceses {(all_packages.license_new != all_packages.license_old).sum()}"
)

In [ ]:
all_packages[all_packages.license_new != all_packages.license_old].head()

In [ ]:
print(
    f"Number of missing from new packages {len(all_packages[all_packages.package_name_new == ""])}"
)

In [ ]:
print(
    f"Number of missing from old packages {len(all_packages[all_packages.package_name_old == ""])}"
)

In [ ]:
mismatches = all_packages.loc[
    all_packages.license_new != all_packages.license_old
].sort_values(by=["package_name_general"])

mismatches.to_csv("mismatches.csv", index=False)

mismatches.loc[mismatches.license_old != ""]

In [ ]:
metas = im.metadata("arrow")
for i in metas.items():
    if i[0] == "Classifier":
        if i[1].startswith("License ::"):
            print(i)